2. Data Acquisition and Exploratory Data Analysis (EDA)

In [ ]:
import os
from tabulate import tabulate

# Path to dataset
dataset_path = r"../data/raw_img/parking"

# Get file list with details
file_details = []
for file in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file)
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert size to MB
    file_ext = os.path.splitext(file)[1]
    
    file_details.append([file, file_ext, f"{file_size:.2f} MB"])

# Create table
print(tabulate(file_details, headers=["Filename", "Extension", "Size"], tablefmt="pretty"))


In [ ]:
import cv2
from icecream import ic

# Since the dataset consists of .mp4 videos, we need to extract frames to analyze the parking lot structure.

# Path to a sample video
video_path = os.path.join(dataset_path, "parking_crop.mp4")

import os
import cv2


def extract_sample_frames(video_path, frame_interval=100, output_dir_name=None):
    """
    Extracts and saves sample frames from a video file.
    
    Parameters:
        video_path (str): Path to the input video file.
        frame_interval (int): Interval at which frames are saved (default is every 100 frames).
        output_dir_name (str, optional): Name of the directory to save extracted frames.
            If not provided, a new directory named after the video file (without extension) 
            will be created in the same directory as the video.
    """
    output_dir = os.path.splitext(video_path)[0]  # Use video name as folder
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_idx in range(0, total_frames, frame_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Jump to frame
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_dir, f"sample_frame_{frame_idx}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved: {frame_filename}")
    cap.release()
    print("Frame extraction complete.")

extract_sample_frames(video_path)

    




In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import cv2
import random
import matplotlib.pyplot as plt

def display_random_frames(frame_directory, num_frames=10):
    """
    Randomly selects 'num_frames' image files from the given directory 
    and displays them in a single combined figure.
    
    Parameters:
        frame_directory (str): Directory containing frame images.
        num_frames (int): Number of random frames to display (default is 10).
    """
    # Get list of image files (considering common image extensions)
    image_files = [f for f in os.listdir(frame_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print("No image files found in the directory.")
        return
    
    # If there are fewer images than required, use all of them
    if len(image_files) < num_frames:
        print(f"Warning: Only {len(image_files)} images found. Displaying all.")
        selected_files = image_files
    else:
        selected_files = random.sample(image_files, num_frames)
    
    # Set up the grid for display: 2 rows and 5 columns (or adjust accordingly)
    cols = 5
    rows = (num_frames + cols - 1) // cols  # ensures sufficient rows
    
    plt.figure(figsize=(15, 6))
    
    for i, image_file in enumerate(selected_files):
        image_path = os.path.join(frame_directory, image_file)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image: {image_file}")
            continue
        # Convert image from BGR (OpenCV) to RGB (matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img_rgb)
        plt.title(image_file, fontsize=8)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

frames_directory = r'..\data\raw_img\parking\parking_crop'
display_random_frames(frames_directory)


### Inspect Parking Lot Mask

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt

# Re-import necessary libraries since execution state was reset
import os
import random
import cv2
import matplotlib.pyplot as plt

def check_mask_alignment_multiple(mask_path, frames_dir, num_frames=5):
    """
    Loads a mask image and compares it to 'num_frames' randomly selected frames from 'frames_dir'.
    Displays each original frame alongside its mask overlay.
    
    Parameters:
        mask_path (str): Path to the mask image (e.g., "mask_1920_1080.png").
        frames_dir (str): Path to the directory containing extracted frames (e.g., "parking_1920_1080").
        num_frames (int): Number of frames to compare against the mask (default is 5).
    """
    # Load the mask in grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Error loading mask: {mask_path}")
        return
    
    # Get a list of image files in the frames directory
    frame_files = [
        f for f in os.listdir(frames_dir)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]
    
    if not frame_files:
        print(f"No frame images found in '{frames_dir}'.")
        return
    
    # Select 'num_frames' randomly from the available frames
    selected_files = random.sample(frame_files, min(num_frames, len(frame_files)))

    # Set up the figure size based on the number of frames
    fig, axes = plt.subplots(len(selected_files), 2, figsize=(12, 6 * len(selected_files)))

    for i, frame_file in enumerate(selected_files):
        frame_path = os.path.join(frames_dir, frame_file)
        
        # Load the selected frame
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Error loading frame: {frame_path}")
            continue
        
        # Resize mask if necessary
        if (mask.shape[0] != frame.shape[0]) or (mask.shape[1] != frame.shape[1]):
            mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
        else:
            mask_resized = mask

        # Create an overlay by blending the frame and mask
        mask_3ch = cv2.cvtColor(mask_resized, cv2.COLOR_GRAY2BGR)
        overlay = cv2.addWeighted(frame, 0.7, mask_3ch, 0.3, 0)

        # Convert BGR to RGB for display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

        # Display each frame with its mask overlay
        if len(selected_files) > 1:
            axes[i, 0].imshow(frame_rgb)
            axes[i, 0].set_title(f"Original Frame: {frame_file}")
            axes[i, 0].axis("off")

            axes[i, 1].imshow(overlay_rgb)
            axes[i, 1].set_title(f"Overlay with Mask: {frame_file}")
            axes[i, 1].axis("off")
        else:
            axes[0].imshow(frame_rgb)
            axes[0].set_title(f"Original Frame: {frame_file}")
            axes[0].axis("off")

            axes[1].imshow(overlay_rgb)
            axes[1].set_title(f"Overlay with Mask: {frame_file}")
            axes[1].axis("off")

    plt.tight_layout()
    plt.show()

check_mask_alignment_multiple(
    r"../data/raw_img/parking/mask_crop.png", 
    r"../data/raw_img/parking/parking_crop", 
    num_frames=5
)


In [ ]:
import torch
import cv2
from icecream import ic
from ultralytics import YOLO

%matplotlib inline

# Check if GPU is available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the YOLOv5 model (yolov5s is a small, fast variant)
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model_path = "yolo11n"  # automatic download if not found


# model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model = YOLO(model_path).to(device)

# # Set confidence threshold 
# model.conf = 0.2

def detect_vehicles_only(image_path, model, confidence_threshold=0.3):
    """
    Detect only certain vehicles (car, truck, bus) above 'confidence_threshold'.
    """
    img = cv2.imread(image_path)
    results = model.predict(img)
    
    # The first item in 'results' is your predictions object
    preds = results[0]
    
    for box in preds.boxes:
        cls_id = int(box.cls[0])        # Class ID (integer)
        conf = float(box.conf[0])      # Confidence score
        class_name = model.names[cls_id]  # e.g. "car", "person", etc.
        
        # Filter by confidence and class name
        if conf > confidence_threshold and class_name in ["car", "bus", "truck"]:
            # Box coordinates in [x1, y1, x2, y2] format
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Draw a rectangle around the detected object
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Optionally put label + confidence
            label = f"{class_name} {conf:.2f}"
            cv2.putText(img, label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display the final image in matplotlib
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()



image_path = os.path.join(dataset_path, "parking_crop", "sample_frame_0.jpg")
detect_vehicles_only(image_path, model, confidence_threshold=0.3)



In [29]:
from ultralytics import solutions

# Create a ParkingPtsSelection instance
selector = solutions.ParkingPtsSelection()



requirements: Ultralytics requirement ['tkinter'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "tkinter" ' returned non-zero exit status 1.


In [ ]:
import os
import cv2
import json
from ultralytics import solutions

# Path to dataset
dataset_path = r"../data/raw_img/parking"
video_path = os.path.join(dataset_path, "parking_crop.mp4")
json_path = "bounding_boxes.json"  # path to parking annotations file

# Open video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("parking_management.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize parking management object - pass the json file directly
management = solutions.ParkingManagement(model_path="yolov8n", json_file=json_path)

# Process the video
while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break
    
    # Process the frame with the ParkingManagement system
    # This will handle detection, tracking, and spot analysis
    im0 = management.process_data(im0)
    
    # Show the result
    cv2.imshow("Parking Management", im0)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    
    video_writer.write(im0)

# Clean up
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print("Processing complete! Output saved to 'parking_management.mp4'")


Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': 'bounding_boxes.json', 'records': 5, 'model_path': 'yolov8n'}

0: 640x544 1 spoon, 1 microwave, 5 ovens, 92.7ms
Speed: 4.7ms preprocess, 92.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 544)
WARNING  no tracks found!

0: 640x544 1 spoon, 1 microwave, 5 ovens, 69.2ms
Speed: 2.6ms preprocess, 69.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 544)
WARNING  no tracks found!

0: 640x544 1 microwave, 4 ovens, 72.0ms
Speed: 2.9ms preprocess, 72.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 544)
WARNING  no tracks found!

0: 640x544 1 microwave, 5 ovens, 72.4ms
Speed: 2.8ms preprocess, 72.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 544)
WARNING  no tracks found!

0: 640x544 1 microwave, 3 ovens, 68.2ms
Speed: 2.9ms preprocess, 68.2ms i

In [3]:
import os
import cv2
import json
import numpy as np
from ultralytics import YOLO

# Path to dataset
dataset_path = r"../data/raw_img/parking"
video_path = os.path.join(dataset_path, "parking_crop.mp4")
json_path = "bounding_boxes.json"  # path to parking annotations file

# Load parking spots from JSON file
with open(json_path, 'r') as f:
    parking_data = json.load(f)
    
# Extract parking spots coordinates
parking_spots = []
for spot in parking_data:
    points = spot['points']
    # Convert the points to numpy array of integers
    polygon_points = np.array(points, np.int32)
    parking_spots.append(polygon_points)

print(f"Extracted {len(parking_spots)} parking spots")

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Open video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("parking_management.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Function to check if a spot is occupied
def is_spot_occupied(spot_polygon, results, confidence_threshold=0.3, iou_threshold=0.2):
    # Create a mask for the parking spot
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask, [spot_polygon], 255)
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Get confidence
            conf = float(box.conf[0])
            if conf < confidence_threshold:
                continue
                
            # Get class ID
            cls_id = int(box.cls[0])
            
            # Only consider cars (class 2), trucks (7) or buses (5)
            if cls_id not in [2, 5, 7]:  # 2=car, 5=bus, 7=truck in COCO
                continue
                
            # Get bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Create a mask for the detection
            detection_mask = np.zeros((h, w), dtype=np.uint8)
            cv2.rectangle(detection_mask, (x1, y1), (x2, y2), 255, -1)
            
            # Calculate intersection
            intersection = cv2.bitwise_and(mask, detection_mask)
            intersection_area = cv2.countNonZero(intersection)
            
            # Calculate IoU
            spot_area = cv2.countNonZero(mask)
            detection_area = cv2.countNonZero(detection_mask)
            
            # Avoid division by zero
            if spot_area + detection_area - intersection_area == 0:
                continue
                
            iou = intersection_area / float(spot_area + detection_area - intersection_area)
            
            if iou > iou_threshold:
                return True
                
    return False

# Process the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run YOLOv8 inference on the frame
    results = model(frame)
    
    # Draw parking spots and check if they're occupied
    occupied_count = 0
    for i, spot in enumerate(parking_spots):
        # Check if spot is occupied
        occupied = is_spot_occupied(spot, results)
        if occupied:
            occupied_count += 1
        
        # Draw the parking spot (green for empty, red for occupied)
        color = (0, 0, 255) if occupied else (0, 255, 0)
        cv2.polylines(frame, [spot], True, color, 2)
        
        # Add spot number
        centroid = np.mean(spot, axis=0, dtype=np.int32)
        cv2.putText(frame, str(i+1), tuple(centroid), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Draw YOLOv8 detections (only vehicles)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Only draw vehicle classes
            cls_id = int(box.cls[0])
            if cls_id in [2, 5, 7]:  # 2=car, 5=bus, 7=truck in COCO
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                class_name = model.names[cls_id]
                label = f"{class_name}: {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # Add some statistics
    empty_count = len(parking_spots) - occupied_count
    total_spots = len(parking_spots)
    cv2.putText(frame, f"Available: {empty_count}/{total_spots}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Show the result
    cv2.imshow("Parking Management", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    
    video_writer.write(frame)

# Clean up
cap.release()
video_writer.release()
cv2.destroyAllWindows()

print("Processing complete! Output saved to 'parking_management.mp4'")

Extracted 14 parking spots

0: 640x544 2 microwaves, 70.6ms
Speed: 3.1ms preprocess, 70.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 2 microwaves, 64.5ms
Speed: 3.3ms preprocess, 64.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 63.3ms
Speed: 2.6ms preprocess, 63.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 60.1ms
Speed: 3.1ms preprocess, 60.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 58.7ms
Speed: 2.8ms preprocess, 58.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 58.1ms
Speed: 2.3ms preprocess, 58.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 59.8ms
Speed: 2.8ms preprocess, 59.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

0: 640x544 (no detections), 57.2ms
Speed: 2.8ms